# Global optimization solvers

# 1. Basin Hopping

## 1.1 Description

This is a Dashboard interface for the <a href="https://docs.scipy.org/doc/scipy-0.18.1/reference/generated/scipy.optimize.basinhopping.html">Basin hopping</a> from the scipy package. 

Basin-hopping is a stochastic algorithm which attempts to find the global minimum of a smooth scalar function of one or more variables [R144] [R145] [R146] [R147]. The algorithm in its current form was described by David Wales and Jonathan Doye [R145] http://www-wales.ch.cam.ac.uk/.

The algorithm is iterative with each cycle composed of the following features

    random perturbation of the coordinates
    local minimization
    accept or reject the new coordinates based on the minimized function value

The acceptance test used here is the Metropolis criterion of standard Monte Carlo algorithms, although there are many other possibilities [R146].

This is a graphical description of how it works:

<img src="solvers_data/bh.png"></img>

## 1.2 Implementation and use

In [18]:
import time#
import pandas as pd
import numpy as np
import random
import copy

from scipy.optimize import  differential_evolution,minimize
from scipy.optimize import basinhopping
from scipy.optimize import  differential_evolution

from IPython.core.display import clear_output

from shaolin.core.dashboard import Dashboard
from test_functions import TestFunctions

In [20]:
class MyBounds(object):
    """Class in charge of managing the boundaries of the sampling region for the basin hopping solver."""
    def __init__(self, xmax=[1.1,1.1], xmin=[-1.1,-1.1],other_test=None ):
        def true(**kwargs):
            return True
        self._outer_accept_test = true if other_test is None else other_test
        self.xmax = np.array(xmax)
        self.xmin = np.array(xmin)

    def __call__(self, **kwargs):
        x = kwargs["x_new"]
        out = self._outer_accept_test(**kwargs)
        tmax = bool(np.all(x <= self.xmax))
        tmin = bool(np.all(x >= self.xmin))
        return tmax and tmin and out

class BasinHopping(Dashboard):
    """Find the global minimum of a function using the basin-hopping algorithm
    
    Parameters
    ----------
    objfunc: ObjectiveFunction object.
        Object representing the objective function to be solved. 
    x0 : ndarray
        Initial guess.
    niter : integer, optional
        The number of basin hopping iterations
    T : float, optional
        The "temperature" parameter for the accept or reject criterion.  Higher
        "temperatures" mean that larger jumps in function value will be
        accepted.  For best results ``T`` should be comparable to the
        separation
        (in function value) between local minima.
    stepsize : float, optional
        initial step size for use in the random displacement.
    minimizer_kwargs : dict, optional
        Extra keyword arguments to be passed to the minimizer
        ``scipy.optimize.minimize()`` Some important options could be:
            method : str
                The minimization method (e.g. ``"L-BFGS-B"``)
            args : tuple
                Extra arguments passed to the objective function (``func``) and
                its derivatives (Jacobian, Hessian).
    take_step : callable ``take_step(x)``, optional
        Replace the default step taking routine with this routine.  The default
        step taking routine is a random displacement of the coordinates, but
        other step taking algorithms may be better for some systems.
        ``take_step`` can optionally have the attribute ``take_step.stepsize``.
        If this attribute exists, then ``basinhopping`` will adjust
        ``take_step.stepsize`` in order to try to optimize the global minimum
        search.
    accept_test : callable, ``accept_test(f_new=f_new, x_new=x_new, f_old=fold, x_old=x_old)``, optional
        Define a test which will be used to judge whether or not to accept the
        step.  This will be used in addition to the Metropolis test based on
        "temperature" ``T``.  The acceptable return values are True,
        False, or ``"force accept"``. If any of the tests return False
        then the step is rejected. If the latter, then this will override any
        other tests in order to accept the step. This can be used, for example,
        to forcefully escape from a local minimum that ``basinhopping`` is
        trapped in.
    callback : callable, ``callback(x, f, accept)``, optional
        A callback function which will be called for all minima found.  ``x``
        and ``f`` are the coordinates and function value of the trial minimum,
        and ``accept`` is whether or not that minimum was accepted.  This can be
        used, for example, to save the lowest N minima found.  Also,
        ``callback`` can be used to specify a user defined stop criterion by
        optionally returning True to stop the ``basinhopping`` routine.
    interval : integer, optional
        interval for how often to update the ``stepsize``
    disp : bool, optional
        Set to True to print status messages
    niter_success : integer, optional
        Stop the run if the global minimum candidate remains the same for this
        number of iterations.
        
        
    Attributes
    ----------
    epoch: int
        Current iteration of the selected solver.
    pot_func: ObjectiveFunction object.
        Object representing the objective function to be solved. 
    best: (aray,float),  (n_components,),1
        Tuple containing the minimum value found for the objective function
        and the parameters that yielded that value.
    best_pos_so_far: (array), (n_components)
        Alias for the best parameters found by the optimizer.
    best_so_far: float
        Alias for minimum value found by the optimizer.
        """
    def __init__(self,
                 objfunc,
                 x0=None,
                 max_reads=10000000,
                 niter=100,
                 T=1.0,
                 stepsize=0.5,
                 minimizer_kwargs=None,
                 take_step=None,
                 accept_test=None,
                 callback=None,
                 interval=50,
                 disp=False,
                 niter_success=None,
                 **kwargs):
        def false(**kwargs):
            return False
        def true(**kwargs):
            return True
        self._outer_accept_test = true if accept_test is None else accept_test
        self.minimizer_kwargs = minimizer_kwargs
        self.take_step = take_step
        
       
        self.pot_func = objfunc
        self.x0 = x0
        self.best_pos_so_far = None
        self.best_so_far = 1e20
        self._ext_callback =lambda x,val,accept: False if callback is None else callback
        self._run_callback = false
        self.epoch=0
        niter_success = 0 if niter_success is None else niter_success
        strats = ['best1bin','best1exp','rand1exp','randtobest1exp','best2exp','rand2exp','randtobest1bin','best2bin','rand2bin','rand1bin']
        dash = ['r$N=differential_evolution',["##Basin-hopping$n=title",
                                              ['r$N=controls_row',[["c$N=first_row",["(0,100000,1,"+str(niter)+")$d=Niter&n=max_epoch",
                                                               "(0,1e8,1,"+str(max_reads)+")$d=Max reads",
                                                               "(0,1e4,1,"+str(int(niter_success))+")$d=niter_success"]],
                                             
                                              ["c$N=sec_row",["(0.,10000.,.01,"+str(T)+")$d=T",
                                                              "(0,1e8,1,"+str(interval)+")$d=Interval",
                                                               "(0.,100.,0.01,"+str(stepsize)+")$d=Stepsize"]
                                              ],
                                              ["c$n=btn_col",["["+str(bool(disp))+"]$d=Disp","btn$d=Run&n=run_btn"]]]
               ]]]
        Dashboard.__init__(self,dash,**kwargs)
        self.run_btn.observe(self.run)
    
    
    def accept_test(self,**kwargs):
        """Custom acceptance test support"""
        out = self._outer_accept_test(**kwargs)
        return out or self.pot_func.in_domain(kwargs['x'])
    
    
    
    @property
    def best(self):
        return (self.best_pos_so_far.copy(), float(self.best_so_far))
    
        
    
    def run(self,_=None, end_callback=None):
        """Solves the objective function"""
        self.reset()
        if not end_callback is None:
            self._run_callback = end_callback
        niter_success = None if self.niter_success.value == 0 else int(self.niter_success.value)
        x0 = self.pot_func.random_in_domain()
        
        xmax = [x[1] for x in self.pot_func.domain]
        xmin = [x[0] for x in self.pot_func.domain]
        
        bounds = MyBounds(xmax=xmax,xmin=xmin,other_test=self._outer_accept_test)
        result = basinhopping(self.pot_func.evaluate,
                              x0,
                              niter=self.max_epoch.value,
                              T=self.t.value,
                              stepsize=self.stepsize.value,
                              minimizer_kwargs=self.minimizer_kwargs,
                              take_step=self.take_step,
                              accept_test=bounds,
                              callback=self.callback,
                              interval=self.interval.value,
                              disp=self.disp.value,
                              niter_success=niter_success)
        
        clear_output(True)
        print(result)
        self.update_best(result.x)

    def callback(self,x,val,accept):
        """custom callback for finishing the optimization in a scipy-like manner."""
        self.update_best(x)
        self.epoch += 1
        end = self._ext_callback(x,val,accept)
        return end or self._run_callback() 
    
    def reset(self):
        """Resets the solver attributes"""
        self.best_pos_so_far = None
        self.best_so_far = 1e20
        self.epoch=0
        self.pot_func.n_reads = 0
    
    def update_best(self,x):
        """Keeps track of the best value and parameters found by the solver"""
        val = self.pot_func.evaluate(x)
        if self.best_so_far>val:
            self.best_pos_so_far = x
            self.best_so_far = val

First, let's select the Levy's function nº13 from our TestFunctions class:

In [13]:
funcs = TestFunctions()

In [25]:
levy13 = funcs.functions['levy13']

Now we can pass it as a parameter for the Dashboard.

In [26]:
bh = BasinHopping(levy13)

In [27]:
bh.widget

                        fun: 1.921671616497636e-16
 lowest_optimization_result:       fun: 1.921671616497636e-16
 hess_inv: array([[  5.58520193e-03,  -7.67311429e-06],
       [ -7.67311429e-06,   4.99999044e-01]])
      jac: array([  1.15770956e-06,  -5.21661010e-09])
  message: 'Optimization terminated successfully.'
     nfev: 40
      nit: 4
     njev: 10
   status: 0
  success: True
        x: array([ 1.        ,  0.99999999])
                    message: ['requested number of basinhopping iterations completed successfully']
      minimization_failures: 0
                       nfev: 5456
                        nit: 100
                       njev: 1364
                          x: array([ 1.        ,  0.99999999])


<img src="solvers_data/bh_widget.png"></img>

You can selct the desired parameters for the algorithm using the widgets, and once you are ready click Run to start the optimization process. Once it has finished the best value found will be displayed. In order to access it you can also access the *best* attribute o the class. 

In [29]:
bh.best

(array([ 1.        ,  0.99999999]), 1.921671616497636e-16)

# 2. Differential evolution

<a href="https://docs.scipy.org/doc/scipy-0.18.1/reference/generated/scipy.optimize.differential_evolution.html">Differential evolution</a> is a stochastic population based method that is useful for global optimization problems. At each pass through the population the algorithm mutates each candidate solution by mixing with other candidate solutions to create a trial candidate. There are several strategies [R151] for creating trial candidates, which suit some problems more than others. The ‘best1bin’ strategy is a good starting point for many systems. In this strategy two members of the population are randomly chosen. Their difference is used to mutate the best member (the best in best1bin), b0, so far:

**b′=b0+mutation∗(population[rand0]−population[rand1])**

A trial vector is then constructed. Starting with a randomly chosen ‘i’th parameter the trial is sequentially filled (in modulo) with parameters from b’ or the original candidate. The choice of whether to use b’ or the original candidate is made with a binomial distribution (the ‘bin’ in ‘best1bin’) - a random number in [0, 1) is generated. If this number is less than the recombination constant then the parameter is loaded from b’, otherwise it is loaded from the original candidate. The final parameter is always loaded from b’. Once the trial candidate is built its fitness is assessed. If the trial is better than the original candidate then it takes its place. If it is also better than the best overall candidate it also replaces that. To improve your chances of finding a global minimum use higher popsize values, with higher mutation and (dithering), but lower recombination values. This has the effect of widening the search radius, but slowing convergence.

In [30]:
class DifferentialEvolution(Dashboard):
    """Shaolin interface for the Differential Evolution solver from the scipy package.
    Parameters
    ----------
    objfunc: ObjectiveFunction object.
        Object representing the objective function to be solved. 
    #This is from scipy.optimize.differential_evolution
    bounds : sequence
        Bounds for variables.  ``(min, max)`` pairs for each element in ``x``,
        defining the lower and upper bounds for the optimizing argument of
        `func`. It is required to have ``len(bounds) == len(x)``.
        ``len(bounds)`` is used to determine the number of parameters in ``x``.
    args : tuple, optional
        Any additional fixed parameters needed to
        completely specify the objective function.
    strategy : str, optional
        The differential evolution strategy to use. Should be one of:
            - 'best1bin'
            - 'best1exp'
            - 'rand1exp'
            - 'randtobest1exp'
            - 'best2exp'
            - 'rand2exp'
            - 'randtobest1bin'
            - 'best2bin'
            - 'rand2bin'
            - 'rand1bin'
        The default is 'best1bin'.
    maxiter : int, optional
        The maximum number of generations over which the entire population is
        evolved. The maximum number of function evaluations (with no polishing)
        is: ``(maxiter + 1) * popsize * len(x)``
    popsize : int, optional
        A multiplier for setting the total population size.  The population has
        ``popsize * len(x)`` individuals.
    tol : float, optional
        When the mean of the population energies, multiplied by tol,
        divided by the standard deviation of the population energies
        is greater than 1 the solving process terminates:
        ``convergence = mean(pop) * tol / stdev(pop) > 1``
    mutation : float or tuple(float, float), optional
        The mutation constant. In the literature this is also known as
        differential weight, being denoted by F.
        If specified as a float it should be in the range [0, 2].
        If specified as a tuple ``(min, max)`` dithering is employed. Dithering
        randomly changes the mutation constant on a generation by generation
        basis. The mutation constant for that generation is taken from
        ``U[min, max)``. Dithering can help speed convergence significantly.
        Increasing the mutation constant increases the search radius, but will
        slow down convergence.
    recombination : float, optional
        The recombination constant, should be in the range [0, 1]. In the
        literature this is also known as the crossover probability, being
        denoted by CR. Increasing this value allows a larger number of mutants
        to progress into the next generation, but at the risk of population
        stability.
    seed : int or `np.random.RandomState`, optional
        If `seed` is not specified the `np.RandomState` singleton is used.
        If `seed` is an int, a new `np.random.RandomState` instance is used,
        seeded with seed.
        If `seed` is already a `np.random.RandomState instance`, then that
        `np.random.RandomState` instance is used.
        Specify `seed` for repeatable minimizations.
    disp : bool, optional
        Display status messages
    callback : callable, `callback(xk, convergence=val)`, optional
        A function to follow the progress of the minimization. ``xk`` is
        the current value of ``x0``. ``val`` represents the fractional
        value of the population convergence.  When ``val`` is greater than one
        the function halts. If callback returns `True`, then the minimization
        is halted (any polishing is still carried out).
    polish : bool, optional
        If True (default), then `scipy.optimize.minimize` with the `L-BFGS-B`
        method is used to polish the best population member at the end, which
        can improve the minimization slightly.
    init : string, optional
        Specify how the population initialization is performed. Should be
        one of:
            - 'latinhypercube'
            - 'random'
        The default is 'latinhypercube'. Latin Hypercube sampling tries to
        maximize coverage of the available parameter space. 'random' initializes
        the population randomly - this has the drawback that clustering can
        occur, preventing the whole of parameter space being covered.
    
    Attributes
    ----------
    epoch: int
        Current iteration of the selected solver.
    pot_func: ObjectiveFunction object.
        Object representing the objective function to be solved. 
    best: (aray,float),  (n_components,),1
        Tuple containing the minimum value found for the objective function
        and the parameters that yielded that value.
    best_pos_so_far: (array), (n_components)
        Alias for the best parameters found by the optimizer.
    best_so_far: float
        Alias for minimum value found by the optimizer.
    """
    def __init__(self,
                 objfunc,
                 strategy='best1bin',
                 maxiter=1000,
                 popsize=15,
                 max_reads=150000,
                 tol=0.01,
                 mutation=(0.5, 1),
                 recombination=0.7,
                 seed=160290,
                 callback=None,
                 disp=False,
                 polish=True,
                 init='latinhypercube',
                 **kwargs):
        
        def false():
            return False
        self.pot_func = objfunc
        self.best_pos_so_far = None
        self.best_so_far = 1e20
        self._ext_callback =lambda x,conv: False if callback is None else callback
        self._run_callback = false
        self.epoch=0
        strats = ['best1bin','best1exp','rand1exp','randtobest1exp','best2exp','rand2exp','randtobest1bin','best2bin','rand2bin','rand1bin']
        dash = ['r$N=differential_evolution',["##Differential evolution$n=title",
                                              ['r$N=controls_row',[["c$N=first_row",["(0,100000,1,"+str(maxiter)+")$d=Maxiter&n=max_epoch",
                                                               "(0,1e8,1,"+str(max_reads)+")$d=Max reads",
                                                               "(1,1e5,1,"+str(int(popsize))+")$d=Popsize"]],
                                             
                                              ["c$N=sec_row",["(0.,10.,.01,"+str(tol)+")$d=Tol",
                                                               "(0.,100.,0.1,"+str(mutation)+")$d=Mutation",
                                                               "(0,1e8,1,"+str(recombination)+")$d=Recombination"]
                                              ],
                                            ["c$N=third_row",["(0,1000000,1,"+str(seed)+")$d=seed",
                                                               "dd$d=Strategy&o="+str(strats)+"&val="+str(strategy),
                                                               "togs$d=Init&o=['latinhypercube','random']",
                                                               ]
                                              ],["c$n=btn_col",["["+str(bool(polish))+"]$d=Polish","btn$d=Run&n=run_btn"]]]
               ]]]
        Dashboard.__init__(self,dash,**kwargs)
        self.run_btn.observe(self.run)
        
    @property
    def best(self):
        return (self.best_pos_so_far.copy(), float(self.best_so_far))
        
    
    def run(self,_=None, end_callback=None):
        """Solves the objective funtion"""
        self.reset()
        if not end_callback is None:
            self._run_callback = end_callback
        result = differential_evolution(self.pot_func.evaluate,
                                        self.pot_func.domain,
                                        callback=self.callback,
                                        maxiter=int(self.max_epoch.value),
                                        popsize=int(self.popsize.value),
                                        tol=self.tol.value,
                                        mutation=self.mutation.value,
                                        recombination=self.recombination.value,
                                        strategy=self.strategy.value,
                                        init=self.init.value,
                                        polish=self.polish.value)
        
        clear_output(True)
        print(result)
        
        self.update_best(result.x)

    def callback(self,x,convergence):
        """Callback support for custom stopping of the function"""
        self.update_best(x)
        self.epoch += 1
        end = self._ext_callback(x,convergence)

        return end or self._run_callback() 
    
    def reset(self):
        """Resets the solver attributes"""
        self.best_pos_so_far = None
        self.best_so_far = 1e20
        self.epoch=0
        self.pot_func.n_reads = 0
    
    def update_best(self,x):
        """Keeps track of the best value and parameters found by the solver"""
        val = self.pot_func.evaluate(x)
        if self.best_so_far>val:
            self.best_pos_so_far = x
            self.best_so_far = val

In [31]:
de = DifferentialEvolution(levy13)

In [33]:
de.widget

     fun: 3.7946734490167239e-21
 message: 'Optimization terminated successfully.'
    nfev: 1653
     nit: 54
 success: True
       x: array([ 1.,  1.])


<img src="solvers_data/de_widget.png"></img>

In [34]:
de.best

(array([ 1.,  1.]), 3.794673449016724e-21)

# 3. Solver selector

No we blended the two dashboards into one, so we are able to choose which algorithm will be use so solve a given objective function.

In [37]:
class Optimizer(Dashboard):
    """Widget interface to scipy global optimization algorithms.
    It allows to select the parameters for a global optimizer.
    
    Supported algorithms are Basin hopping and Diferential evolution. 
    This class executes the solver on a function object.
    
    Attributes
    ----------
    algos : list, ['differential_evolution','basin_hopping']
        Contains the names of the available solvers.
    epoch: int
        Current iteration of the selected solver.
    max_epoch: int
        Maximum number of iterations of the selected solver.
    max_reads: int
        Maximum number of functions reads allowed for the selected solver.
    pot_func: ObjectiveFunction object.
        Object representing the objective function to be solved. 
    best: (aray,float),  (n_components,),1
        tuple containing the minimum value found for the objective function
        and the parameters that yielded that value.
    """
    def __init__(self,objfunc,select='Basin hopping',fractal_kwargs={},basinhopping_kwargs={},diffevo_kwargs={},**kwargs):
        
        differential_evolution = DifferentialEvolution(objfunc,name='differential_evolution',**diffevo_kwargs)
        basin_hopping = BasinHopping(objfunc,name='basin_hopping',**basinhopping_kwargs)

        
        dash = ["c$N=optimizer_dash",[differential_evolution,
                                      basin_hopping,
                                      ["r$n=btn_row",
                                        ["togs$N=algo_sel&o=['Differential evolution','Basin hopping']&val="+str(select),
                                         'btn$d=Run&n=run_btn']
                                      ]
                                     ]
               ]
        
        Dashboard.__init__(self,dash,**kwargs)
        self.algos = ['differential_evolution','basin_hopping']
        self._last_eval = select.lower().replace(' ','_')
        self.algo_sel.observe(self.update_layout)
        self.run_btn.observe(self.run)
        self.update_layout()
    
    def run(self,_=None, name=None, end_callback=None):
        """Solves the objective funtion with the currently selected parameters"""
        opt = self.algo_sel.value.lower().replace(' ','_') if name is None else name
        self._last_eval = opt
        getattr(self,opt).run(end_callback=end_callback)
    
    @property 
    def epoch(self):
        return getattr(self,self._last_eval).epoch
    @epoch.setter
    def epoch(self,val):
        getattr(self,self._last_eval).epoch = val
        
    @property 
    def max_epoch(self):
        return getattr(self,self._last_eval).max_epoch
    @max_epoch.setter
    def max_epoch(self,val):
        getattr(self,self._last_eval).max_epoch = val
        
    @property 
    def max_reads(self):
        return getattr(self,self._last_eval).max_reads
    @max_reads.setter
    def max_reads(self,val):
        getattr(self,self._last_eval).max_reads = val
    
    @property
    def pot_func(self):
        return getattr(self,self._last_eval).pot_func
    
    @pot_func.setter
    def pot_func(self,val):
        getattr(self,self._last_eval).pot_func = val
    
    @property
    def best(self):
        optimizer = getattr(self,self._last_eval)
        return (optimizer.best_pos_so_far, float(optimizer.best_so_far))
    
    def reset(self):
        """Reset the solvers"""
        self.basin_hopping.reset()
        self.differential_evolution.reset()
    
    def update_layout(self,_=None):
        """widget interface updating"""
        if self.algo_sel.value == "Differential evolution":
            self.differential_evolution.visible = True
            self.differential_evolution.run_btn.visible = False
            self.basin_hopping.visible = False
          
        elif self.algo_sel.value == "Basin hopping":
            self.differential_evolution.visible = False
            self.basin_hopping.visible = True
            self.basin_hopping.run_btn.visible = False


In [38]:
solvers = Optimizer(levy13)

In [39]:
solvers.widget

                        fun: 9.531626804523418e-17
 lowest_optimization_result:       fun: 9.531626804523418e-17
 hess_inv: array([[  5.46483894e-03,  -7.99089436e-05],
       [ -7.99089436e-05,   4.99906990e-01]])
      jac: array([  1.15863571e-06,   1.03142188e-08])
  message: 'Optimization terminated successfully.'
     nfev: 48
      nit: 8
     njev: 12
   status: 0
  success: True
        x: array([ 1.,  1.])
                    message: ['requested number of basinhopping iterations completed successfully']
      minimization_failures: 0
                       nfev: 5672
                        nit: 100
                       njev: 1418
                          x: array([ 1.,  1.])


<img src="solvers_data/de_widget.png"></img>

In [41]:
solvers.best

(array([ 1.,  1.]), 9.531626804523418e-17)

In [1]:
%%file optimizer.py
import time#
import pandas as pd
import numpy as np
import random
import copy

from scipy.optimize import  differential_evolution,minimize
from scipy.optimize import basinhopping
from scipy.optimize import  differential_evolution

from IPython.core.display import clear_output

from shaolin.core.dashboard import Dashboard
from test_functions import TestFunctions

"""
Created on Tue May 24 13:13:28 2016

@author: Hossam Faris
"""
import math
import numpy
import random
import time


class solution:

    def __init__(self):
        self.endTime=None
        self.executionTime=None
        self.convergence=None
        self.optimizer=None
        self.objfname=None
        self.fmin = None
        self.nest = None
            
def get_cuckoos(nest,best,domain,n,dim):
    
    # perform Levy flights
    tempnest=numpy.zeros((n,dim))
    tempnest=numpy.array(nest)
    beta=3/2;
    sigma=(math.gamma(1+beta)*math.sin(math.pi*beta/2)/(math.gamma((1+beta)/2)*beta*2**((beta-1)/2)))**(1/beta);

    s=numpy.zeros(dim)
    for j in range (0,n):
        s=nest[j,:]
        u=numpy.random.randn(len(s))*sigma
        v=numpy.random.randn(len(s))
        step=u/abs(v)**(1/beta)
 
        stepsize=0.01*(step*(s-best))

        s=s+stepsize*numpy.random.randn(len(s))
    
        for i,(lb,ub) in enumerate(domain):
            tempnest[j,i]=numpy.clip(s[i], lb, ub)
    return tempnest

def get_best_nest(nest,newnest,fitness,n,dim,objf):
# Evaluating all new solutions
    tempnest=numpy.zeros((n,dim))
    tempnest=numpy.copy(nest)

    for j in range(0,n):
    #for j=1:size(nest,1),
   
        fnew=objf.evaluate(newnest[j,:])
        #print(fnew)
        if fnew<=fitness[j] and objf.in_domain(newnest[j,:]):
           fitness[j]=fnew
           tempnest[j,:]=newnest[j,:]
        
    # Find the current best

    fmin = min(fitness)
    K=numpy.argmin(fitness)
    bestlocal=tempnest[K,:]

    return fmin,bestlocal,tempnest,fitness

# Replace some nests by constructing new solutions/nests
def empty_nests(nest,pa,n,dim):

    # Discovered or not 
    tempnest=numpy.zeros((n,dim))

    K=numpy.random.uniform(0,1,(n,dim))>pa
    
    
    stepsize=random.random()*(nest[numpy.random.permutation(n),:]-nest[numpy.random.permutation(n),:])

    
    tempnest=nest+stepsize*K
 
    return tempnest
##########################################################################


def CS(objf,lb,ub,dim,n,N_IterTotal,callback):

    pa=0.25
    nd=dim
    convergence=[]
    # RInitialize nests randomely
    nest=numpy.random.rand(n,dim)*(ub-lb)+lb
    new_nest=numpy.zeros((n,dim))
    new_nest=numpy.copy(nest)
    bestnest=[0]*dim;
    fitness=numpy.zeros(n) 
    fitness.fill(float("inf"))
    s=solution()
    timerStart=time.time() 
    s.startTime=time.strftime("%Y-%m-%d-%H-%M-%S")

    fmin,bestnest,nest,fitness =get_best_nest(nest,new_nest,fitness,n,dim,objf)
    convergence = [];
    # Main loop counter
    iter = 0
    while iter <= N_IterTotal and not callback() and (fmin-objf.benchmark[1])>10**-6:
    # Generate new solutions (but keep the current best
        new_nest=get_cuckoos(nest,bestnest,objf.domain,n,dim)
        # Evaluate new solutions and find best
        fnew,best,nest,fitness=get_best_nest(nest,new_nest,fitness,n,dim,objf)
        new_nest=empty_nests(new_nest,pa,n,dim) ;
        # Evaluate new solutions and find best
        fnew,best,nest,fitness=get_best_nest(nest,new_nest,fitness,n,dim,objf)
        if (iter%100==0):
            result = minimize(objf.evaluate,
                              bestnest,
                              bounds=objf.domain,
                              method="L-BFGS-B",
                             )
            best = result.x
            fnew = result.fun
        if fnew<fmin and objf.in_domain(best):
            fmin=fnew
            bestnest=best
            s.fmin = fmin
            s.nest = bestnest
        convergence.append(fmin)
        iter += 1

    timerEnd=time.time()  
    s.endTime=time.strftime("%Y-%m-%d-%H-%M-%S")
    s.executionTime=timerEnd-timerStart
    s.convergence=convergence
    s.optimizer="CS"
    s.objfname=objf.evaluate.__name__
    s.fmin = fmin
    s.nest = bestnest
    print("best value found: {}".format(fmin))
    return s

from shaolin import KungFu
import numpy as np
class CuckooSearch(KungFu):
    
    def __init__(self,objfunc,num_iters=500,n_birds=50,callback=None,max_reads=50000,**kwargs):
        
        self.num_iters = num_iters
        self.n_birds = n_birds
        self.pot_func = objfunc
        def false():
            return False
        self.pot_func = objfunc
        #self.best_pos_so_far = None
        #self.best_so_far = 1e20
        self._ext_callback =lambda x,conv: False if callback is None else callback
        self._run_callback = false
        self.epoch=0
        self.solution = solution()
        self.solution.nest = np.zeros(len(self.pot_func.random_in_domain()))
        self.solution.fmin = 1e20
        KungFu.__init__(self,**{'title':'#Cuckoo search$N=title&d=',
                         'num_iters':(0,1000000,1,num_iters),
                         'max_reads':(0,1000000,1,max_reads),
                         'n_birds':(0,1000000,1,n_birds),
                         'run_button':'@btn$N=run_btn&d=Run'
                        },box='1r|',**kwargs)
    
    @property
    def best(self):
        return self.solution.nest,self.solution.fmin
    @property
    def best_pos_so_far(self):
        return self.solution.nest
    @property
    def best_so_far(self):
        return self.solution.fmin
    
    def reset(self):
        self.solution.nest = np.zeros(len(self.pot_func.random_in_domain()))
        self.solution.fmin = 1e20
        self.epoch=0
        self.pot_func.n_reads = 0
    
    def run(self,_=None, end_callback=None):
        #print('runnin')
        #self.solution.nest = np.zeros(len(self.pot_func.random_in_domain()))
        dim = len(self.pot_func.random_in_domain())
        ub = np.asarray(self.pot_func.domain).max()
        lb = np.asarray(self.pot_func.domain).min()
        
        self.solution = CS(self.pot_func,lb,ub,dim,self.n_birds(),self.num_iters(),end_callback)
        
    def callback(self,x,convergence):
        """Callback support for custom stopping of the function"""
        self.epoch += 1
        end = self._ext_callback(x,convergence)

        return end or self._run_callback() 

class Optimizer(Dashboard):
    """Widget interface to scipy global optimization algorithms.
    It allows to select the parameters for a global optimizer.
    
    Supported algorithms are Basin hopping and Diferential evolution. 
    This class executes the solver on a function object.
    
    Attributes
    ----------
    algos : list, ['differential_evolution','basin_hopping']
        Contains the names of the available solvers.
    epoch: int
        Current iteration of the selected solver.
    max_epoch: int
        Maximum number of iterations of the selected solver.
    max_reads: int
        Maximum number of functions reads allowed for the selected solver.
    pot_func: ObjectiveFunction object.
        Object representing the objective function to be solved. 
    best: (aray,float),  (n_components,),1
        tuple containing the minimum value found for the objective function
        and the parameters that yielded that value.
    """
    def __init__(self,objfunc,select='Cuckoo search',fractal_kwargs={},basinhopping_kwargs={},diffevo_kwargs={},**kwargs):
        
        differential_evolution = DifferentialEvolution(objfunc,name='differential_evolution',**diffevo_kwargs)
        basin_hopping = BasinHopping(objfunc,name='basin_hopping',**basinhopping_kwargs)
        cuckoo = CuckooSearch(objfunc=objfunc,name='cuckoo_search')
        
        dash = ["c$N=optimizer_dash",[differential_evolution,
                                      basin_hopping,
                                      cuckoo,
                                      ["r$n=btn_row",
                                        ["togs$N=algo_sel&o=['Cuckoo search','Differential evolution','Basin hopping']&val="+str(select),
                                         'btn$d=Run&n=run_btn']
                                      ]
                                     ]
               ]
        
        Dashboard.__init__(self,dash,**kwargs)
        self.algos = ['cuckoo_search','differential_evolution','basin_hopping']
        self._last_eval = select.lower().replace(' ','_')
        self.algo_sel.observe(self.update_layout)
        self.run_btn.observe(self.run)
        #self.cuckoo_search.run_btn.observe(self.run)
        self.update_layout()
    
    def run(self,_=None, name=None, end_callback=None):
        """Solves the objective funtion with the currently selected parameters"""
        opt = self.algo_sel.value.lower().replace(' ','_') if name is None else name
        self._last_eval = opt
        getattr(self,opt).run(end_callback=end_callback)
    
    @property 
    def epoch(self):
        return getattr(self,self._last_eval).epoch
    @epoch.setter
    def epoch(self,val):
        getattr(self,self._last_eval).epoch = val
        
    @property 
    def max_epoch(self):
        return getattr(self,self._last_eval).max_epoch
    @max_epoch.setter
    def max_epoch(self,val):
        getattr(self,self._last_eval).max_epoch = val
        
    @property 
    def max_reads(self):
        return getattr(self,self._last_eval).max_reads
    @max_reads.setter
    def max_reads(self,val):
        getattr(self,self._last_eval).max_reads = val
    
    @property
    def pot_func(self):
        return getattr(self,self._last_eval).pot_func
    
    @pot_func.setter
    def pot_func(self,val):
        getattr(self,self._last_eval).pot_func = val
    
    @property
    def best(self):
        optimizer = getattr(self,self._last_eval)
        return (optimizer.best_pos_so_far, float(optimizer.best_so_far))
    
    def reset(self):
        """Reset the solvers"""
        self.basin_hopping.reset()
        self.differential_evolution.reset()
        self.cuckoo_search.reset()
    
    def update_layout(self,_=None):
        """widget interface updating"""
        if self.algo_sel.value == "Differential evolution":
            self.differential_evolution.visible = True
            self.differential_evolution.run_btn.visible = True
            self.basin_hopping.visible = False
            self.basin_hopping.run_btn.visible = False
            self.cuckoo_search.run_btn.visible = False
            self.cuckoo_search.visible = False
          
        elif self.algo_sel.value == "Basin hopping":
            self.differential_evolution.visible = False
            self.basin_hopping.visible = True
            self.basin_hopping.run_btn.visible = True
            self.cuckoo_search.run_btn.visible = False
            self.cuckoo_search.visible = False
            
        elif self.algo_sel.value == 'Cuckoo search':
            self.differential_evolution.visible = False
            self.differential_evolution.run_btn.visible = False
            self.basin_hopping.visible = False
            self.basin_hopping.run_btn.visible = False
            self.cuckoo_search.run_btn.visible = False
            self.cuckoo_search.visible = True


class DifferentialEvolution(Dashboard):
    """Shaolin interface for the Differential Evolution solver from the scipy package.
    Parameters
    ----------
    objfunc: ObjectiveFunction object.
        Object representing the objective function to be solved. 
    #This is from scipy.optimize.differential_evolution
    bounds : sequence
        Bounds for variables.  ``(min, max)`` pairs for each element in ``x``,
        defining the lower and upper bounds for the optimizing argument of
        `func`. It is required to have ``len(bounds) == len(x)``.
        ``len(bounds)`` is used to determine the number of parameters in ``x``.
    args : tuple, optional
        Any additional fixed parameters needed to
        completely specify the objective function.
    strategy : str, optional
        The differential evolution strategy to use. Should be one of:
            - 'best1bin'
            - 'best1exp'
            - 'rand1exp'
            - 'randtobest1exp'
            - 'best2exp'
            - 'rand2exp'
            - 'randtobest1bin'
            - 'best2bin'
            - 'rand2bin'
            - 'rand1bin'
        The default is 'best1bin'.
    maxiter : int, optional
        The maximum number of generations over which the entire population is
        evolved. The maximum number of function evaluations (with no polishing)
        is: ``(maxiter + 1) * popsize * len(x)``
    popsize : int, optional
        A multiplier for setting the total population size.  The population has
        ``popsize * len(x)`` individuals.
    tol : float, optional
        When the mean of the population energies, multiplied by tol,
        divided by the standard deviation of the population energies
        is greater than 1 the solving process terminates:
        ``convergence = mean(pop) * tol / stdev(pop) > 1``
    mutation : float or tuple(float, float), optional
        The mutation constant. In the literature this is also known as
        differential weight, being denoted by F.
        If specified as a float it should be in the range [0, 2].
        If specified as a tuple ``(min, max)`` dithering is employed. Dithering
        randomly changes the mutation constant on a generation by generation
        basis. The mutation constant for that generation is taken from
        ``U[min, max)``. Dithering can help speed convergence significantly.
        Increasing the mutation constant increases the search radius, but will
        slow down convergence.
    recombination : float, optional
        The recombination constant, should be in the range [0, 1]. In the
        literature this is also known as the crossover probability, being
        denoted by CR. Increasing this value allows a larger number of mutants
        to progress into the next generation, but at the risk of population
        stability.
    seed : int or `np.random.RandomState`, optional
        If `seed` is not specified the `np.RandomState` singleton is used.
        If `seed` is an int, a new `np.random.RandomState` instance is used,
        seeded with seed.
        If `seed` is already a `np.random.RandomState instance`, then that
        `np.random.RandomState` instance is used.
        Specify `seed` for repeatable minimizations.
    disp : bool, optional
        Display status messages
    callback : callable, `callback(xk, convergence=val)`, optional
        A function to follow the progress of the minimization. ``xk`` is
        the current value of ``x0``. ``val`` represents the fractional
        value of the population convergence.  When ``val`` is greater than one
        the function halts. If callback returns `True`, then the minimization
        is halted (any polishing is still carried out).
    polish : bool, optional
        If True (default), then `scipy.optimize.minimize` with the `L-BFGS-B`
        method is used to polish the best population member at the end, which
        can improve the minimization slightly.
    init : string, optional
        Specify how the population initialization is performed. Should be
        one of:
            - 'latinhypercube'
            - 'random'
        The default is 'latinhypercube'. Latin Hypercube sampling tries to
        maximize coverage of the available parameter space. 'random' initializes
        the population randomly - this has the drawback that clustering can
        occur, preventing the whole of parameter space being covered.
    
    Attributes
    ----------
    epoch: int
        Current iteration of the selected solver.
    pot_func: ObjectiveFunction object.
        Object representing the objective function to be solved. 
    best: (aray,float),  (n_components,),1
        Tuple containing the minimum value found for the objective function
        and the parameters that yielded that value.
    best_pos_so_far: (array), (n_components)
        Alias for the best parameters found by the optimizer.
    best_so_far: float
        Alias for minimum value found by the optimizer.
    """
    def __init__(self,
                 objfunc,
                 strategy='best1bin',
                 maxiter=1000,
                 popsize=15,
                 max_reads=150000,
                 tol=0.01,
                 mutation=(0.5, 1),
                 recombination=0.7,
                 seed=160290,
                 callback=None,
                 disp=False,
                 polish=True,
                 init='latinhypercube',
                 **kwargs):
        
        def false():
            return False
        self.pot_func = objfunc
        self.best_pos_so_far = None
        self.best_so_far = 1e20
        self._ext_callback =lambda x,conv: False if callback is None else callback
        self._run_callback = false
        self.epoch=0
        strats = ['best1bin','best1exp','rand1exp','randtobest1exp','best2exp','rand2exp','randtobest1bin','best2bin','rand2bin','rand1bin']
        dash = ['r$N=differential_evolution',["##Differential evolution$n=title",
                                              ['r$N=controls_row',[["c$N=first_row",["(0,100000,1,"+str(maxiter)+")$d=Maxiter&n=max_epoch",
                                                               "(0,1e8,1,"+str(max_reads)+")$d=Max reads",
                                                               "(1,1e5,1,"+str(int(popsize))+")$d=Popsize"]],
                                             
                                              ["c$N=sec_row",["(0.,10.,.01,"+str(tol)+")$d=Tol",
                                                               "(0.,100.,0.1,"+str(mutation)+")$d=Mutation",
                                                               "(0,1e8,1,"+str(recombination)+")$d=Recombination"]
                                              ],
                                            ["c$N=third_row",["(0,1000000,1,"+str(seed)+")$d=seed",
                                                               "dd$d=Strategy&o="+str(strats)+"&val="+str(strategy),
                                                               "togs$d=Init&o=['latinhypercube','random']",
                                                               ]
                                              ],["c$n=btn_col",["["+str(bool(polish))+"]$d=Polish","btn$d=Run&n=run_btn"]]]
               ]]]
        Dashboard.__init__(self,dash,**kwargs)
        self.run_btn.observe(self.run)
        
    @property
    def best(self):
        return (self.best_pos_so_far.copy(), float(self.best_so_far))
        
    
    def run(self,_=None, end_callback=None):
        """Solves the objective funtion"""
        self.reset()
        if not end_callback is None:
            self._run_callback = end_callback
        result = differential_evolution(self.pot_func.evaluate,
                                        self.pot_func.domain,
                                        callback=self.callback,
                                        maxiter=int(self.max_epoch.value),
                                        popsize=int(self.popsize.value),
                                        tol=self.tol.value,
                                        mutation=self.mutation.value,
                                        recombination=self.recombination.value,
                                        strategy=self.strategy.value,
                                        init=self.init.value,
                                        polish=self.polish.value)
        
        clear_output(True)
        print(result)
        
        self.update_best(result.x)

    def callback(self,x,convergence):
        """Callback support for custom stopping of the function"""
        self.update_best(x)
        self.epoch += 1
        end = self._ext_callback(x,convergence)

        return end or self._run_callback() 
    
    def reset(self):
        """Resets the solver attributes"""
        self.best_pos_so_far = None
        self.best_so_far = 1e20
        self.epoch=0
        self.pot_func.n_reads = 0
    
    def update_best(self,x):
        """Keeps track of the best value and parameters found by the solver"""
        val = self.pot_func.evaluate(x)
        if self.best_so_far>val:
            self.best_pos_so_far = x
            self.best_so_far = val

class MyBounds(object):
    """Class in charge of managing the boundaries of the sampling region for the basin hopping solver."""
    def __init__(self, xmax=[1.1,1.1], xmin=[-1.1,-1.1],other_test=None ):
        def true(**kwargs):
            return True
        self._outer_accept_test = true if other_test is None else other_test
        self.xmax = np.array(xmax)
        self.xmin = np.array(xmin)

    def __call__(self, **kwargs):
        x = kwargs["x_new"]
        out = self._outer_accept_test(**kwargs)
        tmax = bool(np.all(x <= self.xmax))
        tmin = bool(np.all(x >= self.xmin))
        return tmax and tmin and out

class BasinHopping(Dashboard):
    """Find the global minimum of a function using the basin-hopping algorithm
    
    Parameters
    ----------
    objfunc: ObjectiveFunction object.
        Object representing the objective function to be solved. 
    x0 : ndarray
        Initial guess.
    niter : integer, optional
        The number of basin hopping iterations
    T : float, optional
        The "temperature" parameter for the accept or reject criterion.  Higher
        "temperatures" mean that larger jumps in function value will be
        accepted.  For best results ``T`` should be comparable to the
        separation
        (in function value) between local minima.
    stepsize : float, optional
        initial step size for use in the random displacement.
    minimizer_kwargs : dict, optional
        Extra keyword arguments to be passed to the minimizer
        ``scipy.optimize.minimize()`` Some important options could be:
            method : str
                The minimization method (e.g. ``"L-BFGS-B"``)
            args : tuple
                Extra arguments passed to the objective function (``func``) and
                its derivatives (Jacobian, Hessian).
    take_step : callable ``take_step(x)``, optional
        Replace the default step taking routine with this routine.  The default
        step taking routine is a random displacement of the coordinates, but
        other step taking algorithms may be better for some systems.
        ``take_step`` can optionally have the attribute ``take_step.stepsize``.
        If this attribute exists, then ``basinhopping`` will adjust
        ``take_step.stepsize`` in order to try to optimize the global minimum
        search.
    accept_test : callable, ``accept_test(f_new=f_new, x_new=x_new, f_old=fold, x_old=x_old)``, optional
        Define a test which will be used to judge whether or not to accept the
        step.  This will be used in addition to the Metropolis test based on
        "temperature" ``T``.  The acceptable return values are True,
        False, or ``"force accept"``. If any of the tests return False
        then the step is rejected. If the latter, then this will override any
        other tests in order to accept the step. This can be used, for example,
        to forcefully escape from a local minimum that ``basinhopping`` is
        trapped in.
    callback : callable, ``callback(x, f, accept)``, optional
        A callback function which will be called for all minima found.  ``x``
        and ``f`` are the coordinates and function value of the trial minimum,
        and ``accept`` is whether or not that minimum was accepted.  This can be
        used, for example, to save the lowest N minima found.  Also,
        ``callback`` can be used to specify a user defined stop criterion by
        optionally returning True to stop the ``basinhopping`` routine.
    interval : integer, optional
        interval for how often to update the ``stepsize``
    disp : bool, optional
        Set to True to print status messages
    niter_success : integer, optional
        Stop the run if the global minimum candidate remains the same for this
        number of iterations.
        
        
    Attributes
    ----------
    epoch: int
        Current iteration of the selected solver.
    pot_func: ObjectiveFunction object.
        Object representing the objective function to be solved. 
    best: (aray,float),  (n_components,),1
        Tuple containing the minimum value found for the objective function
        and the parameters that yielded that value.
    best_pos_so_far: (array), (n_components)
        Alias for the best parameters found by the optimizer.
    best_so_far: float
        Alias for minimum value found by the optimizer.
        """
    def __init__(self,
                 objfunc,
                 x0=None,
                 max_reads=10000000,
                 niter=100,
                 T=1.0,
                 stepsize=0.5,
                 minimizer_kwargs=None,
                 take_step=None,
                 accept_test=None,
                 callback=None,
                 interval=50,
                 disp=False,
                 niter_success=None,
                 **kwargs):
        def false(**kwargs):
            return False
        def true(**kwargs):
            return True
        self._outer_accept_test = true if accept_test is None else accept_test
        self.minimizer_kwargs = minimizer_kwargs
        self.take_step = take_step
        
       
        self.pot_func = objfunc
        self.x0 = x0
        self.best_pos_so_far = None
        self.best_so_far = 1e20
        self._ext_callback =lambda x,val,accept: False if callback is None else callback
        self._run_callback = false
        self.epoch=0
        niter_success = 0 if niter_success is None else niter_success
        strats = ['best1bin','best1exp','rand1exp','randtobest1exp','best2exp','rand2exp','randtobest1bin','best2bin','rand2bin','rand1bin']
        dash = ['r$N=differential_evolution',["##Basin-hopping$n=title",
                                              ['r$N=controls_row',[["c$N=first_row",["(0,100000,1,"+str(niter)+")$d=Niter&n=max_epoch",
                                                               "(0,1e8,1,"+str(max_reads)+")$d=Max reads",
                                                               "(0,1e4,1,"+str(int(niter_success))+")$d=niter_success"]],
                                             
                                              ["c$N=sec_row",["(0.,10000.,.01,"+str(T)+")$d=T",
                                                              "(0,1e8,1,"+str(interval)+")$d=Interval",
                                                               "(0.,100.,0.01,"+str(stepsize)+")$d=Stepsize"]
                                              ],
                                              ["c$n=btn_col",["["+str(bool(disp))+"]$d=Disp","btn$d=Run&n=run_btn"]]]
               ]]]
        Dashboard.__init__(self,dash,**kwargs)
        self.run_btn.observe(self.run)
    
    
    def accept_test(self,**kwargs):
        """Custom acceptance test support"""
        out = self._outer_accept_test(**kwargs)
        return out or self.pot_func.in_domain(kwargs['x'])
    
    
    
    @property
    def best(self):
        return (self.best_pos_so_far.copy(), float(self.best_so_far))
    
        
    
    def run(self,_=None, end_callback=None):
        """Solves the objective function"""
        self.reset()
        if not end_callback is None:
            self._run_callback = end_callback
        niter_success = None if self.niter_success.value == 0 else int(self.niter_success.value)
        x0 = self.pot_func.random_in_domain()
        
        xmax = [x[1] for x in self.pot_func.domain]
        xmin = [x[0] for x in self.pot_func.domain]
        
        bounds = MyBounds(xmax=xmax,xmin=xmin,other_test=self._outer_accept_test)
        result = basinhopping(self.pot_func.evaluate,
                              x0,
                              niter=self.max_epoch.value,
                              T=self.t.value,
                              stepsize=self.stepsize.value,
                              minimizer_kwargs=self.minimizer_kwargs,
                              take_step=self.take_step,
                              accept_test=bounds,
                              callback=self.callback,
                              interval=self.interval.value,
                              disp=self.disp.value,
                              niter_success=niter_success)
        
        clear_output(True)
        print(result)
        self.update_best(result.x)

    def callback(self,x,val,accept):
        """custom callback for finishing the optimization in a scipy-like manner."""
        self.update_best(x)
        self.epoch += 1
        end = self._ext_callback(x,val,accept)
        return end or self._run_callback() 
    
    def reset(self):
        """Resets the solver attributes"""
        self.best_pos_so_far = None
        self.best_so_far = 1e20
        self.epoch=0
        self.pot_func.n_reads = 0
    
    def update_best(self,x):
        """Keeps track of the best value and parameters found by the solver"""
        val = self.pot_func.evaluate(x)
        if self.best_so_far>val:
            self.best_pos_so_far = x
            self.best_so_far = val

Overwriting optimizer.py
